# Examples with Dataframes

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## To Dataframes

Write the Poisson data of a partition to a dataframe.

## From Dataframes

Get Poisson data partition from a dataframe.

## Rolling vector aggregations

Evaluate the rolling vector aggregations.

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"

	"gonum.org/v1/gonum/floats"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

func diffFunc(
	params *simulator.Params,
	partitionIndex int,
	stateHistories []*simulator.StateHistory,
	timestepsHistory *simulator.CumulativeTimestepsHistory,
) []float64 {
	values := make([]float64, stateHistories[partitionIndex].StateWidth)
	floats.SubTo(values, params.Get("data"), stateHistories[int(
		params.GetIndex("partition", 0))].Values.RawRowView(0))
	return values
}

%%

storage := analysis.NewStateTimeStorageFromPartitions(
	[]*simulator.PartitionConfig{
		{
			Name:              "poisson_data",
			Iteration:         &discrete.PoissonProcessIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"rates": {0.002, 0.001, 0.004, 0.001},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              123,
		},
		{
			Name:              "diff_poisson_data",
			Iteration:         &general.ValuesFunctionIteration{Function: diffFunc},
			Params:            simulator.NewParams(make(map[string][]float64)),
			ParamsAsPartitions: map[string][]string{
				"partition": {"poisson_data"},
			},
			ParamsFromUpstream: map[string]simulator.NamedUpstreamConfig{
				"data": {Upstream: "poisson_data"},
			},
			InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              0,
	    },
    },
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	1667980544.0,
)
meanPartition := analysis.NewVectorMeanPartition(
	analysis.AppliedAggregation{
		Name: "diffs_mean",
		Data: analysis.DataRef{PartitionName: "diff_poisson_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	},
	storage,
)
meanPartition.Params.Set("exponential_weighting_timescale", []float64{70000.0})
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	meanPartition,
	map[string]int{"diff_poisson_data": 100},
)
variancePartition := analysis.NewVectorVariancePartition(
	analysis.DataRef{PartitionName: "diffs_mean"},
	analysis.AppliedAggregation{
		Name: "diffs_variance",
		Data: analysis.DataRef{PartitionName: "diff_poisson_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	},
	storage,
)
variancePartition.Params.Set("exponential_weighting_timescale", []float64{70000.0})
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	variancePartition,
	map[string]int{"diffs_mean": 1, "diff_poisson_data": 100},
)

xRef := analysis.DataRef{IsTime: true}
yRefs := []analysis.DataRef{{PartitionName: "poisson_data"}}
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

yRefs = []analysis.DataRef{{
	PartitionName: "diffs_mean",
	TimeRange: &analysis.IndexRange{Lower: 100, Upper: 1000},
}}
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)
line.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")

yRefs = []analysis.DataRef{{
	PartitionName: "diffs_variance",
	TimeRange: &analysis.IndexRange{Lower: 100, Upper: 1000},
}}
line = analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)
line.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")